# Question 7

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torch.optim import Adam

# Set random seed for reproducibility
torch.manual_seed(42)

# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    
])

batch_size = 16
train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = MNIST(root='./data', train=False, download=True, transform=transform)

train_size = 50000
val_size = 10000 #len(train) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [ ]:
def cnn(lr,main_epochs):
    val_loss_arr = []
    val_acc_arr = []
    # Define the CNN model
    class SimpleCNN(nn.Module):
        def __init__(self):
            super(SimpleCNN, self).__init__()
            self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
            self.relu1 = nn.ReLU()
            self.maxpool1 = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
            self.relu2 = nn.ReLU()
            self.maxpool2 = nn.MaxPool2d(kernel_size=2)
            self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
            self.relu3 = nn.ReLU()
            self.maxpool3 = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()
            self.fc = nn.Linear(64 * 3 * 3, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.maxpool1(x)
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.maxpool2(x)
            x = self.conv3(x)
            x = self.relu3(x)
            x = self.maxpool3(x)
            x = self.flatten(x)
            x = self.fc(x)
            return x

    # Instantiate the model
    model = SimpleCNN()

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    # Training loop
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(main_epochs):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            output = model(data)

            # Calculate loss
            loss = criterion(output, target)

            # Backward pass
            loss.backward()

            # Update weights
            optimizer.step()

            # Print training statistics
            if batch_idx % 100 == 0:
                print(f"Epoch {epoch+1}/{main_epochs}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item()}")

        # Validation loop
        model.eval()
        val_loss = 0.0
        correct = 0

        with torch.no_grad():
            for data, target in val_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                val_loss += criterion(output, target).item()
                _, predicted = output.max(1)
                correct += predicted.eq(target).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct / len(val_loader.dataset) * 100.0

        print(f"Validation Loss: {val_loss}, Accuracy: {accuracy}%")
        val_acc_arr.append(accuracy)
        val_loss_arr.append(val_loss)

    print("Training finished.")
    return val_loss_arr, val_acc_arr

In [ ]:
cnn(0.001,10)

# Question 8 

In [ ]:
lr = [0.001,0.003,0.01,0.03]
main_epochs = [5,10]
ma_dict = {}

for i in lr:
    for j in main_epochs:
        print("Learning Rate: ",i)
        print("Epochs: ",j)
        val_loss_arr, val_acc_arr = cnn(i,j)
        print("Validation Loss Array: ",val_loss_arr)
        print("Validation Accuracy Array: ",val_acc_arr)
        ma_dict[(i,j)] = (val_acc_arr,val_loss_arr)

# Question 9

In [ ]:
# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])